# Save raw data to 60-20-20 split for train-test-validation sets

In [1]:
import time
import pathlib
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import when, col, to_date, date_trunc, rank, monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DateType, FloatType
from pyspark.context import SparkContext

# spark.stop()

In [2]:
SparkContext().setCheckpointDir(dirName="/cephfs/train_test_val/_checkpoint")

conf = pyspark.SparkConf().setAll([\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/16 23:48:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.3.1'

### Structs we might need

In [3]:
glucose_data_schema=StructType([StructField('PatientId', StringType(), True),
                                StructField('Value', FloatType(), True),
                                StructField('GlucoseDisplayTime', TimestampType(), True),
                                StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                StructField('GlucoseDisplayDate', DateType(), True),
                                StructField('NumId', IntegerType(), True)
                                ])

raw_schema=StructType([StructField('_c0', IntegerType(),True),
                                StructField('PostDate', TimestampType(),True),
                                StructField('IngestionDate', TimestampType(),True),
                                StructField('PostId', StringType(),True),
                                StructField('PostTime', TimestampType(), True),
                                StructField('PatientId', StringType(), True),
                                StructField('Stream', StringType(), True),
                                StructField('SequenceNumber', StringType(), True),
                                StructField('TransmitterNumber', StringType(), True),
                                StructField('ReceiverNumber', StringType(), True),
                                StructField('RecordedSystemTime', TimestampType(), True),
                                StructField('RecordedDisplayTime', TimestampType(), True),
                                StructField('RecordedDisplayTimeRaw', TimestampType(), True),
                                StructField('TransmitterId', StringType(), True),
                                StructField('TransmitterTime', StringType(), True),
                                StructField('GlucoseSystemTime', TimestampType(), True),
                                StructField('GlucoseDisplayTime', TimestampType(), True),
                                StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                StructField('Value', FloatType(), True),
                                StructField('Status', StringType(), True),
                                StructField('TrendArrow', StringType(), True),
                                StructField('TrendRate', FloatType(), True),
                                StructField('IsBackFilled', StringType(), True),
                                StructField('InternalStatus', StringType(), True),
                                StructField('SessionStartTime', StringType(), True)])

cohortSchema = StructType([StructField('', IntegerType(), True),
                        StructField('UserId', StringType(), True),
                        StructField('Gender', StringType(), True),
                        StructField('DOB', TimestampType(), True),
                        StructField('Age', IntegerType(), True),
                        StructField('DiabetesType', StringType(), True),
                        StructField('Treatment', StringType(), True)
                        ])

### Get all paths in to read from

In [4]:
'''all parquets of the patient data'''
# allPaths = [str(x) for x in list(pathlib.Path('/cephfs/data_by_patient').glob('*.parquet')) if 'part-00' in str(x)]
allPaths = [str(x) for x in list(pathlib.Path('/cephfs/data').glob('*.csv')) if 'glucose_records' in str(x)]

allPaths.sort()

# print("train length:", len(trainPaths), "\nvalidation length:", len(valPaths),"\ntest length:", len(testPaths))

# print(allPaths[-1])
# allPaths = allPaths[:-1]
len(allPaths)

365

### Read in the Cohort dataframe

In [5]:
# read in cohort dataframe, with Number ID properly labeled
startTime = time.time()

cohortDf = spark.read.options(delimiter=',')\
        .csv('/cephfs/data/cohort.csv', header=True, schema=cohortSchema)\
        .withColumnRenamed('', 'NumId')

print(cohortDf.dtypes)
cohortDf.show(10)
print(time.time() - startTime)

[('NumId', 'int'), ('UserId', 'string'), ('Gender', 'string'), ('DOB', 'timestamp'), ('Age', 'int'), ('DiabetesType', 'string'), ('Treatment', 'string')]


+-----+--------------------+------+-------------------+---+------------+---------+
|NumId|              UserId|Gender|                DOB|Age|DiabetesType|Treatment|
+-----+--------------------+------+-------------------+---+------------+---------+
|    0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|
|    1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|
|    2|uhsyLhr4Zl6NfGbNB...|Female|1938-01-01 00:00:00| 85|    type-two|       no|
|    3|9uAVHBOgoCJ9hfcrL...|  Male|1938-01-01 00:00:00| 85|    type-two|       no|
|    4|Fyb156jU1edGykL7N...|Female|1939-01-01 00:00:00| 84|    type-two|       no|
|    5|86XfZ0fNI0VWOzWrl...|Female|1939-01-01 00:00:00| 84|    type-two|       no|
|    6|JfJMH1qCpiYNuPOp/...|Female|1940-01-01 00:00:00| 83|    type-two|       no|
|    7|EkW0PD80req7mL/5S...|  Male|1940-01-01 00:00:00| 83|    type-two|       no|
|    8|OyqSKorAj1OPZaevj...|Female|1941-01-01 00:00:00| 82|    type-two|       no|
|   

In [6]:
# make mini dataframe of the string IDs and number IDs
patientIds = cohortDf.select(col('UserId'), col('NumId')).distinct()

### Load in the raw data

In [7]:
startTime = time.time()

# df = spark.read \
#            .schema(glucose_data_schema) \
#            .format('parquet') \
#            .load(allPaths)
df = spark.read\
    .format('csv')\
    .option('delimiter', ',')\
    .option("mode", "DROPMALFORMED")\
    .option("header", True)\
    .schema(raw_schema)\
    .load(allPaths)\
    .select(col("PatientId"), col("Value"), \
            col("GlucoseDisplayTime"))

print(time.time() - startTime)

1.92893648147583


In [ ]:
startTime = time.time()

df.printSchema()

print(time.time() - startTime)

root
 |-- PatientId: string (nullable = true)
 |-- Value: float (nullable = true)
 |-- GlucoseDisplayTime: timestamp (nullable = true)

0.001371622085571289


In [ ]:
startTime = time.time()

patIds = [i.PatientId for i in df.select('PatientId').distinct().collect()]
print(len(patIds), "total patients")
print("row count: ", df.count())
df.show()

print(time.time() - startTime)

5977 total patients


row count:  3323456
+--------------------+-----+--------------------+
|           PatientId|Value|  GlucoseDisplayTime|
+--------------------+-----+--------------------+
|1Jxgxke6R3Uh2c9aR...|  0.0|2022-02-01 14:45:...|
|toBStbTTYI2GU28Yd...|  0.0|2022-02-01 17:46:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 14:58:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 22:53:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 22:38:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 05:03:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 09:33:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 16:38:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 20:58:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 23:29:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 03:48:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 11:28:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 20:08:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 10:43:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 06:43:...|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 02:13:...|
|+XAhHhm+BkhqusxsZ...|  0.0|20

### Clean-up

In [8]:
# """the following cell is leslie's cleanup code, yoinked from fill-missing to read-data to here"""
startTime = time.time()

'''get rid of any dates from before the actual start-date of Feb 1, 2022'''
df = df.where("GlucoseDisplayTime > '2022-01-31 23:59:59'")

'''replace 0s with NaN and dropna'''
df = df.withColumn("Value", when(col("Value")=="0", None) \
                                                       .otherwise(col("Value")))
df = df.na.drop(subset=['PatientId','Value','GlucoseDisplayTime'])
# df = df.where(df.Value>0)

df = df.withColumn("GlucoseDisplayTime",
                   date_trunc("minute",
                   col("GlucoseDisplayTime")))

'''drop duplicate datetimes for each patient'''
window = Window.partitionBy('GlucoseDisplayTime','PatientId').orderBy('tiebreak')
df = (df
 .withColumn('tiebreak', monotonically_increasing_id())
 .withColumn('rank', rank().over(window))
 .filter(col('rank') == 1).drop('rank','tiebreak')
)

print(time.time() - startTime)

0.2772250175476074


### **CHECKPOINT**

In [ ]:
startTime = time.time()

df = df.checkpoint()

print(time.time() - startTime)

### Add in (join) the NumId column

In [9]:
'''add numId to df'''
startTime = time.time()

df = df.join(patientIds, df.PatientId == patientIds.UserId)\
            .select(df.PatientId, patientIds.NumId, df.Value, df.GlucoseDisplayTime)

print(time.time() - startTime)

0.08810687065124512


## Split into Train-Test-Val groups of 60-20-20

plan:
* get total count of values per patient
* make 2 cols:
    * int that's the total number of rows that should make up 60% of the patient's data
    * int that's the total number of rows that should make up 80% of the patient's data
* merge that dataframe back onto the main df dataframe
* (is it possible to filter based on another column?)
* add ranks based on patient ID
* filter once that splits at the 60% mark, asking for 'rank'< or > to the 60% mark
    * save the <60% as training
* filter the >60% again on the 80% mark
    * save the <80% as validation
    * save the >80% as test

somewhere in there, get rid of patients with less than 80% data

In [10]:
'''get total counts of values per patient'''
counter = df.groupBy('NumId').count()
# print("row count: ", counter.count())

'''(?) filter out patients with too little usable data'''
minUsable = 0.80 * (60/5 * 24 * 365)
# temp = counter.filter(col('count') < minUsable)
print("the minimum number of glucose recordings we should be allowing are", minUsable)
# print("but there are only", temp.count(), "patients that have at least that much.")
# counter.write.parquet('temp_data/')

'''get the numbers that indicate where the splits between train-val and val-test will be'''
counter = counter.withColumn("split60",(col("count")* 0.6).cast("Integer"))
counter = counter.withColumn("split80",(col("count")* 0.8).cast("Integer"))
counter = counter.drop('count')

# '''if i want to merge both small dataframes first before merging on the big main df'''
# patientIds = patientIds.join(counter, patientIds.UserId == counter.PatientId)\
#             .select(patientIds.NumId, patientIds.UserId, counter.split60, counter.split80)

'''rename for future merge (will get an "ambiguous" error without this)'''
counter = counter.withColumnRenamed("NumId","UserId")

the minimum number of glucose recordings we should be allowing are 84096.0


In [11]:
'''get everything into order for ranking/sorting by 60%-20%-20%'''
startTime = time.time()

df = df.join(counter, df.NumId == counter.UserId)\
            .select(df.PatientId, df.NumId, df.Value, df.GlucoseDisplayTime, \
                    counter.split60, counter.split80)

df = df.orderBy("NumId", "GlucoseDisplayTime", ascending=True)

print(time.time() - startTime)

0.1132810115814209


In [12]:
'''rank, then next cell is filtering'''
startTime = time.time()

window = Window.partitionBy('PatientId').orderBy('tiebreak')
df = df \
 .withColumn('tiebreak', monotonically_increasing_id()) \
 .withColumn('rank', rank().over(window))

print(time.time() - startTime)

0.058053016662597656


In [13]:
'''training set'''
trainSet = df.filter(col('rank') <= col('split60')) \
                 .drop('rank','tiebreak','split60','split80')

'''validation set'''
valSet = df.filter((col('rank') > col('split60')) & (col('rank') <= col('split80'))) \
               .drop('rank','tiebreak','split60','split80')

'''test set'''
testSet = df.filter(col('rank') > col('split80')) \
                .drop('rank','tiebreak','split60','split80')

## Save out into parquet files

In [15]:
'''godspeed'''
startTime = time.time()

# df.repartition('PatientId')\
#     .write.parquet('/cephfs/train_test_val/test/') 
trainSet.repartition('PatientId').write.parquet('/cephfs/train_test_val/train_set/') 
# valSet.repartition('PatientId').write.parquet('/cephfs/train_test_val/val_set/') 
# testSet.repartition('PatientId').write.parquet('/cephfs/train_test_val/test_set/') 

[1119.353s][warning][gc,alloc] Executor task launch worker for task 5.0 in stage 8.0 (TID 6001): Retried waiting for GCLocker too often allocating 131074 words
[1119.353s][warning][gc,alloc] Executor task launch worker for task 2.0 in stage 8.0 (TID 5998): Retried waiting for GCLocker too often allocating 131074 words
[1119.562s][warning][gc,alloc] Executor task launch worker for task 1.0 in stage 8.0 (TID 5997): Retried waiting for GCLocker too often allocating 131074 words
[1119.572s][warning][gc,alloc] Executor task launch worker for task 8.0 in stage 8.0 (TID 6004): Retried waiting for GCLocker too often allocating 131074 words
[1119.572s][warning][gc,alloc] Executor task launch worker for task 7.0 in stage 8.0 (TID 6003): Retried waiting for GCLocker too often allocating 131074 words
[1119.572s][warning][gc,alloc] Executor task launch worker for task 11.0 in stage 8.0 (TID 6007): Retried waiting for GCLocker too often allocating 131074 words
[1119.572s][warning][gc,alloc] Executor

23/05/16 23:18:14 ERROR FileFormatWriter: Aborting job 414e7339-6f64-4779-88d6-28382fb4a3a3.
org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 8.0 failed 1 times, most recent failure: Lost task 8.0 in stage 8.0 (TID 6004) (jupyter-ljoe-40ucsd-2eedu executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler

Py4JJavaError: An error occurred while calling o200.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org

In [ ]:
endTime = time.time()
endTime-startTime

In [16]:
# endTime = time.time()
# endTime-startTime

5.854722222222223

In [15]:
17300/60/60

4.805555555555555

In [16]:
45887/60/60

12.74638888888889